In [5]:
%reload_ext autoreload
%autoreload 2
import sys
from dotenv import load_dotenv
import os
load_dotenv()

ROOT = os.getenv("ROOT")
sys.path.append(ROOT)
from src.data_loader import load_reviews
from src.processing import reviews_processing
from src.nlp.classifier import BERTClassifier

In [6]:
df = load_reviews(category="All_beauty", frac=0.01)
df = reviews_processing(df=df, clean_text=False)
sub = df.rename(columns={"rating": "label", "review_input": "text"})[["label", "text"]]

In [7]:
classifier = BERTClassifier(df=sub)

Map: 100%|██████████| 702/702 [00:00<00:00, 10974.04 examples/s]


[11:54:41] | INFO  | split : {'train_set': 5612, 'eval_set': 701, 'test_set': 702}                 ]8;id=50631;file:///Users/lucanyckees/Desktop/my-repos/recommendation_system/src/nlp/classifier.py\classifier.py]8;;\:]8;id=75954;file:///Users/lucanyckees/Desktop/my-repos/recommendation_system/src/nlp/classifier.py#91\91]8;;\

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
classifier.train()

In [ ]:
classifier.evaluate()

In [ ]:
classifier.predict()